In [1]:
import tensorflow as tf

In [4]:
inputs = tf.keras.Input(shape=[32, 32, 3])
x = tf.keras.layers.Conv2D(10, kernel_size=[3, 3], strides=[1, 1], padding='SAME', activation='relu', name='conv_1')(inputs)
x = tf.keras.layers.AveragePooling2D(pool_size=[2, 2], strides=[2, 2])(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)

In [5]:
model = tf.keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 10)        280       
_________________________________________________________________
average_pooling2d_2 (Average (None, 16, 16, 10)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 10)        40        
_________________________________________________________________
flatten_2 (Flatten)          (None, 2560)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1311232   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130  

## MobileNet V1:移动端网络
- 轻量级卷积神经网络
- 更少的参数、更少的计算量，却有不俗的性能
- 空间可分离卷积

In [10]:
!pip install opencv-python

  Using cached opencv_python-4.2.0.32-cp36-cp36m-win_amd64.whl (33.0 MB)


In [12]:
import numpy as np
import cv2

In [18]:
def mobelnet_v1(
    inputs,
    classes
):
    x = conv_block(inputs, 32, strides=(2,2))
    x = depthwise_conv_block(x, 64)
    x = depthwise_conv_block(x, 128, strides=(2,2))
    x = depthwise_conv_block(x,128)
    x = depthwise_conv_block(x, 256, strides=(2,2))
    x = depthwise_conv_block(x, 256)
    x = depthwise_conv_block(x, 512, strides=(2,2))
    x = depthwise_conv_block(x, 512)
    x = depthwise_conv_block(x, 512)
    x = depthwise_conv_block(x, 512)
    x = depthwise_conv_block(x, 512)
    x = depthwise_conv_block(x, 512)
    x = depthwise_conv_block(x, 1024, strides=(2,2))
    x = depthwise_conv_block(x, 1024)
    
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    pred = tf.keras.layers.Dense(classes, activation='softmax')(x)
    
    return pred


In [14]:
def conv_block(
        inputs,
        filters,
        kernel_size=(3,3),
        strides=(1,1)
):
    x = tf.keras.layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU(6,0)(x)
        

In [20]:
def depthwise_conv_block(
        inputs,
        pointwise_conv_filters,
        strides=(1,1)
):
    x = tf.keras.layers.DepthwiseConv2D((3,3), padding='same', strides=strides, use_bias=False)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(6.0)(x)
    
    x = tf.keras.layers.Conv2D(pointwise_conv_filters, kernel_size=(1,1), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    return tf.keras.layers.ReLU(6.0)(x)

In [21]:
inputs = tf.keras.layers.Input(shape=(32, 32, 3))

model = tf.keras.Model(inputs=inputs, outputs=mobelnet_v1(inputs, 10))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        864       
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 16, 16, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 16, 16, 32)        288       
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 16, 16, 32)        0   

In [24]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 39s 0us/step


In [25]:
print(x_train.shape)

(50000, 32, 32, 3)


In [26]:
x_train = x_train / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)

In [29]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['categorical_accuracy', 'Recall', 'AUC']
)

In [30]:
model.fit(x_train, y_train, batch_size=8, epochs=1)

Train on 50000 samples
10112/50000 [=====>........................] - ETA: 44:42 - loss: 2.3823 - categorical_accuracy: 0.1724 - Recall: 0.0143 - AUC: 0.6255

KeyboardInterrupt: 

In [31]:
model.save('mobilenet_v1_cifar10.h5')

In [35]:
model = tf.keras.models.load_model('mobilenet_v1_cifar10.h5')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        864       
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 16, 16, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 16, 16, 32)        288       
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 16, 16, 32)        0   

In [40]:
img = cv2.imread('ship.jpg', 1)/255.0
img = np.expand_dims(img, 0 )

In [41]:
pred = model.predict(img)

In [42]:
print(pred)

[[0.04090989 0.34230298 0.04489311 0.01345758 0.00380524 0.02520222
  0.01396965 0.02976966 0.03512364 0.45056614]]
